# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import logging
import pandas
import azureml.core
import os
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Workspace, Experiment, Model
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, LocalWebservice, Webservice
import requests
import json

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:

ws = Workspace.from_config()

print (ws.name,ws.resource_group, ws.location,ws.subscription_id, sep = '\n')




quick-starts-ws-137266
aml-quickstarts-137266
southcentralus
cdbe0b43-92a0-4715-838a-f2648cc7ad21


In [3]:
# choose a name for experiment
experiment_name = 'Capstone-aml'

experiment=Experiment(ws, experiment_name)
run= experiment.start_logging()

In [4]:
aml_compute_cluster_name= 'Project3-aml'
try:
    compute_target = ComputeTarget(workspace=ws, name=aml_compute_cluster_name)
    print('Found existing compute target {}.'.format(aml_compute_cluster_name))
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_DS12_V2",
                                                               max_nodes=4)

      # create the cluster
    compute_target = ComputeTarget.create(ws, aml_compute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output= True)
print("Azure Machine Learning Compute attached")

Creating a new compute target...
Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
Azure Machine Learning Compute attached


## Data Import

In [5]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 

datapath="https://archive.ics.uci.edu/ml/machine-learning-databases/00451/dataR2.csv"
ds=TabularDatasetFactory.from_delimited_files(path=datapath)

df=ds.to_pandas_dataframe()

In [6]:
df.head()

,Age,BMI,Glucose,Insulin,HOMA,Leptin,Adiponectin,Resistin,MCP.1,Classification
0,48,23.500000,70,2.707,0.467409,8.8071,9.702400,7.99585,417.114,1
1,83,20.690495,92,3.115,0.706897,8.8438,5.429285,4.06405,468.786,1
2,82,23.124670,91,4.498,1.009651,17.9393,22.432040,9.27715,554.697,1
3,68,21.367521,77,3.226,0.612725,9.8827,7.169560,12.76600,928.220,1
4,86,21.111111,92,3.549,0.805386,6.6994,4.819240,10.57635,773.920,1


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
# TODO: Put your automl settings here

automl_settings = {
    "experiment_timeout_minutes": 15,
    "max_concurrent_iterations": 10,
    "primary_metric" : 'AUC_weighted'
}

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    debug_log='automl_errors.log',
    task='classification',
    training_data= ds,
    label_column_name= 'Classification',
    n_cross_validations=5,
    compute_target=compute_target,
    iterations=100,
    **automl_settings
   )

In [8]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output= True)

Running on remote.
No run_configuration provided, running on Project3-aml with default configuration
Running on remote compute: Project3-aml
Parent Run ID: AutoML_18577a07-5c38-491d-bb1b-6985f60f5edd

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_18577a07-5c38-491d-bb1b-6985f60f5edd',
 'target': 'Project3-aml',
 'status': 'Completed',
 'startTimeUtc': '2021-02-04T23:13:10.163829Z',
 'endTimeUtc': '2021-02-04T23:38:38.654509Z',
 'properties': {'num_iterations': '100',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'Project3-aml',
  'AMLSettingsJsonString': '{"path":null,"name":"Capstone-aml","subscription_id":"cdbe0b43-92a0-4715-838a-f2648cc7ad21","resource_group":"aml-quickstarts-137266","workspace_name":"quick-starts-ws-137266","region":"southcentralus","compute_target":"Project3-aml","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":100,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes"

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [31]:
best_run, best_model = remote_run.get_output()
print(best_model)

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               reg_alpha=0.10416666666666667,
                                                                                               reg_lambda=2.0833333333333335,
                                                                                               scale_pos_weight=1,
                         

In [32]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
Capstone-aml,AutoML_18577a07-5c38-491d-bb1b-6985f60f5edd_54,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [42]:
#TODO: Save the best model
import joblib
joblib.dump(best_model, 'best_automl_model.pkl')

['best_automl_model.pkl']

In [35]:

#print best parameters and accuracy

print("Best Run Id: ", best_run.id)
print('\n AUC:', best_run.get_metrics()['AUC_weighted'])
print("Best model : {}".format(best_model), sep = '\n')
print("\n Parameters: ", best_run.get_details())

Best Run Id:  AutoML_18577a07-5c38-491d-bb1b-6985f60f5edd_54

 AUC: 0.8483660783660782
Best model : Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               reg_alpha=0.10416666666666667,
                                                                                               reg_lambda=2.0833333333333335,
                                        

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [36]:
from azureml.core.model import Model
model = Model.register(workspace = ws, model_name = 'best_automl_model', model_path = 'best_automl_model.pkl')
print(model.name, model.id, model.version, sep='\t')

Registering model best_automl_model
best_automl_model	best_automl_model:2	2


In [37]:
model

Model(workspace=Workspace.create(name='quick-starts-ws-137266', subscription_id='cdbe0b43-92a0-4715-838a-f2648cc7ad21', resource_group='aml-quickstarts-137266'), name=best_automl_model, id=best_automl_model:2, version=2, tags={}, properties={})

In [23]:
##best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'Score.py')

In [39]:
from azureml.core.conda_dependencies import CondaDependencies
import azureml.train.automl

# Create the environment
env = best_run.get_environment()

inference_config = InferenceConfig(entry_script='Score.py', environment=env)

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4, enable_app_insights=True)
webservice = Model.deploy(ws,
                          "cancer-classification", 
                          [model], 
                          inference_config, 
                          deployment_config)
webservice.wait_for_deployment(show_output = True)

print(webservice.state)
print(webservice.scoring_uri)
print(webservice.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://eb4de2bf-6b10-452e-8904-786074bac5cb.southcentralus.azurecontainer.io/score
http://eb4de2bf-6b10-452e-8904-786074bac5cb.southcentralus.azurecontainer.io/swagger.json


In [40]:
# URL for the web service
scoring_uri = 'http://eb4de2bf-6b10-452e-8904-786074bac5cb.southcentralus.azurecontainer.io/score'

# Set the content type
headers = {'Content-Type': 'application/json'}

# Two sets of data to score, so we get two results back
data={'data':[{"Age":50, "BMI":29.5,"Glucose":90,"Insulin":1.907,"HOMA":0.89408667,"Leptin":6.5071,"Adiponectin":8.8024,"Resistin":9.00585,"MCP.1":7.114}]}

# Convert to JSON string
input_data = json.dumps(data)

# Make the request and display the response
response = requests.post(scoring_uri, input_data, headers=headers)

print("Response Code : ", response.status_code)
print("Predicted Value : ",response.text)


Response Code :  200
Predicted Value :  [1]


In [41]:
# Web Service Logs
print(webservice.get_logs())

2021-02-05T00:11:08,081828697+00:00 - gunicorn/run 
2021-02-05T00:11:08,083689719+00:00 - rsyslog/run 
2021-02-05T00:11:08,085492740+00:00 - nginx/run 
2021-02-05T00:11:08,086918657+00:00 - iot-server/run 
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [44]:
webservice.delete()

No service with name cancer-classification found to delete.
